In [1]:
%pip install pandas numpy matplotlib yfinance PyPortfolioOpt

Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
from pypfopt import EfficientFrontier, risk_models, black_litterman, expected_returns, BlackLittermanModel, HRPOpt, CLA

In [3]:
tickers = ["BN", "ENPH", "FSLR"]

In [4]:
ohlc = yf.download(tickers, start='2022-01-01',end='2024-01-01')
prices = ohlc["Adj Close"]
prices.head()

[*********************100%%**********************]  3 of 3 completed


Ticker,BN,ENPH,FSLR
Date,,,
2022-01-03,47.336147,184.449997,88.580002
2022-01-04,47.786217,178.279999,87.279999
2022-01-05,45.622730,157.199997,83.510002
2022-01-06,45.441128,151.490005,83.970001
2022-01-07,45.298996,145.130005,84.680000


In [5]:
market_prices = yf.download("SPY", start='2022-01-01',end='2024-01-01')["Adj Close"]
market_prices.head()

[*********************100%%**********************]  1 of 1 completed


Date
2022-01-03    460.127289
2022-01-04    459.973236
2022-01-05    451.140747
2022-01-06    450.716919
2022-01-07    448.935028
Name: Adj Close, dtype: float64

In [6]:
S = risk_models.CovarianceShrinkage(prices).ledoit_wolf()
delta = black_litterman.market_implied_risk_aversion(market_prices)
delta

0.32037972864784986

Reading in the data; preparing expected returns and a risk model

In [7]:
returns = prices.pct_change().dropna()
mu = expected_returns.mean_historical_return(prices)
returns.head()

Ticker,BN,ENPH,FSLR
Date,,,
2022-01-04,0.009508,-0.033451,-0.014676
2022-01-05,-0.045274,-0.118241,-0.043194
2022-01-06,-0.003981,-0.036323,0.005508
2022-01-07,-0.003128,-0.041983,0.008455
2022-01-10,-0.012027,0.013781,-0.004487


Now try with a nonconvex objective from  Kolm et al (2014)

In [8]:
def deviation_risk_parity(w, cov_matrix):
    diff = w * np.dot(cov_matrix, w) - (w * np.dot(cov_matrix, w)).reshape(-1, 1)
    return (diff**2).sum().sum()

In [9]:
ef = EfficientFrontier(mu, S)
weights = ef.nonconvex_objective(deviation_risk_parity, ef.cov_matrix)
ef.portfolio_performance(verbose=True)

Expected annual return: 4.4%
Annual volatility: 37.4%
Sharpe Ratio: 0.06


(0.043993024000617165, 0.3739744568917449, 0.0641568523156181)

# Black-Litterman

In [10]:
mcaps = {}
for t in tickers:
    stock = yf.Ticker(t)
    mcaps[t] = stock.info["marketCap"]
mcaps

{'BN': 73399001088, 'ENPH': 16126729216, 'FSLR': 24248532992}

In [11]:
prior = black_litterman.market_implied_prior_returns(mcaps, delta, S)

In [12]:
viewdict = {
    "BN": 0.10, #"BN", "ENPH", "FSLR"
    "ENPH": 0.30,
    "FSLR": 0.05,
}

'''
views = np.array([-0.20, 0.10, 0.15]).reshape(-1, 1)
picking = np.array(
    [
        [0, 0, 1],
        [1, 0, -1],
        [0, -1, 1],
    ]
)
bl = BlackLittermanModel(S, Q=views, P=picking, pi=prior, tau=0.01)
'''
bl = BlackLittermanModel(S, pi=prior, absolute_views=viewdict)
rets = bl.bl_returns()
ef = EfficientFrontier(rets, S)
ef.max_sharpe()
print(ef.clean_weights())
ef.portfolio_performance(verbose=True)

OrderedDict([('BN', 0.52257), ('ENPH', 0.47743), ('FSLR', 0.0)])
Expected annual return: 13.0%
Annual volatility: 42.8%
Sharpe Ratio: 0.26


(0.13048678467686317, 0.42849759997278447, 0.25784691602445525)

# Hierarchical risk parity

In [13]:
hrp = HRPOpt(returns)
weights = hrp.optimize()
hrp.portfolio_performance(verbose=True)
print(weights)

import pypfopt.plotting as plotting
plotting.plot_dendrogram(hrp)  # to plot dendrogram

Expected annual return: 7.2%
Annual volatility: 33.6%
Sharpe Ratio: 0.16
OrderedDict([('BN', 0.7063331021130563), ('ENPH', 0.11260492066508086), ('FSLR', 0.1810619772218628)])


/Users/lisirui/anaconda3/envs/bioenv/lib/python3.9/site-packages/pypfopt/hierarchical_portfolio.py:138: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.7063331]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  w[first_cluster] *= alpha  # weight 1


OSError: 'seaborn-deep' is not a valid package style, path of style file, URL of style file, or library style name (library styles are listed in `style.available`)

# Crticial Line Algorithm

In [ ]:
cla = CLA(mu, S)
print(cla.max_sharpe())
cla.portfolio_performance(verbose=True)
plotting.plot_efficient_frontier(cla)  # to plot

OrderedDict([('BN', 0.0), ('ENPH', 0.0), ('FSLR', 1.0)])
Expected annual return: 39.8%
Annual volatility: 52.9%
Sharpe Ratio: 0.71


NameError: name 'plotting' is not defined